<a href="https://colab.research.google.com/github/dittysan/machinelearningpractice/blob/main/Ch3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>